# Financial Chatbot Prototype

This chatbot answers financial questions about Microsoft, Tesla, and Apple from 2023 to 2025. It uses fuzzy matching, context tracking, and trend analysis to respond intelligently.

## Features
- Total revenue calculation
- Company-specific metrics
- Glossary explanations
- Trend summaries
- Personality-based responses

## How to Run
1. Open `advanced_financial_chatbot.py` in Python
2. Make sure `financial_data.csv` is in the same folder
3. Run the script and start chatting

## Limitations
- Only supports 3 companies and 3 years
- Static data only
- No GUI or web interface

## Files Included
- `FinancialChatbot_Prototype.ipynb`
- `financial_data.csv`
- `README.txt`


In [2]:
import pandas as pd
from rapidfuzz import process

# Load and clean financial data
df = pd.read_csv("financial_data.csv")

# Clean numeric columns
numeric_columns = ["Revenue", "Net Income", "Assets", "Liabilities", "Cash Flow"]
for col in numeric_columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace("$", "")
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Initialize context
context = {
    "company": None,
    "year": None,
    "metric": None,
    "history": []
}

# Personality mode
personality = "Analyst"  # Options: Analyst, Intern, Nerd

# Fuzzy matching
def fuzzy_match(user_input, options):
    match, score, _ = process.extractOne(user_input, options)
    return match if score > 60 else None

# Entity extraction
def extract_entities(user_input):
    companies = ["Apple", "Tesla", "Microsoft"]
    metrics = ["Revenue", "Net Income", "Assets", "Liabilities", "Cash Flow", "Profit Margin"]
    years = ["2023", "2024", "2025"]

    company = fuzzy_match(user_input, companies)
    metric = fuzzy_match(user_input, metrics)
    year = fuzzy_match(user_input, years)
    year = int(year) if year else None

    return company, year, metric

# Update context
def update_context(company, year, metric):
    context["company"] = company
    context["year"] = year
    context["metric"] = metric
    context["history"].append((company, year, metric))

# Glossary
def explain_term(term):
    glossary = {
        "Revenue": "Total income from sales before expenses.",
        "Net Income": "Profit after all costs and taxes.",
        "Assets": "Everything the company owns.",
        "Liabilities": "Everything the company owes.",
        "Cash Flow": "Money moving in and out of the business.",
        "Profit Margin": "Percentage of revenue that becomes profit."
    }
    return glossary.get(term.title(), "I'm still learning that one!")

# Trend description
def describe_trend(company, metric):
    data = df[df["Company"] == company].sort_values("Year")
    trend = data[metric].diff().mean()
    direction = "increased" if trend > 0 else "decreased"
    return f"{company}'s {metric.lower()} has generally {direction} over the years."

# Style response
def style_response(text):
    if personality == "Intern":
        return f"Hey! So here's the scoop: {text} 😊"
    elif personality == "Nerd":
        return f"Data alert! 📊 {text} Let’s crunch more numbers!"
    else:
        return f"{text} Let me know if you want deeper insights."

# Generate financial response
def generate_response(company, year, metric):
    try:
        value = df[(df["Company"] == company) & (df["Year"] == year)][metric].values[0]
        update_context(company, year, metric)
        return f"In {year}, {company} reported a {metric.lower()} of ${value:,.0f}."
    except:
        return "Hmm, I couldn’t find that data. Try asking about revenue, net income, assets, liabilities, or cash flow."

# Main chatbot logic
def financial_chatbot(user_input):
    user_input = user_input.lower().strip()

    if "total revenue" in user_input:
        total = df["Revenue"].sum()
        return style_response(f"The total revenue across all companies from 2023 to 2025 is approximately ${total:,.0f}.")

    if "explain" in user_input:
        term = fuzzy_match(user_input, list(df.columns))
        return style_response(explain_term(term))

    if "trend" in user_input:
        company, _, metric = extract_entities(user_input)
        if company and metric:
            return style_response(describe_trend(company, metric))

    if "compare" in user_input and context["company"] and context["metric"]:
        return f"Sure — which company or year would you like to compare {context['company']}'s {context['metric']} to?"

    company, year, metric = extract_entities(user_input)
    if company and year and metric:
        return style_response(generate_response(company, year, metric))

    return style_response(
        "I’m here to help with financial insights. Try asking something like:\n"
        "- 'What was Tesla’s revenue in 2025?'\n"
        "- 'Compare Apple and Microsoft’s net income in 2024.'\n"
        "- 'Explain profit margin.'"
    )

# Chat loop
print("👋 Hi! I’m your financial assistant. Ask me anything about Microsoft, Tesla, or Apple from 2023 to 2025.")
print("Type 'exit' to end our chat.\n")

while True:
    user_input = input("💬 You: ")
    if user_input.lower() == "exit":
        print("👋 Bot: Catch you later!")
        break
    print("🤖 Bot:", financial_chatbot(user_input), "\n")

👋 Hi! I’m your financial assistant. Ask me anything about Microsoft, Tesla, or Apple from 2023 to 2025.
Type 'exit' to end our chat.



💬 You:  what was the total revenue


🤖 Bot: The total revenue across all companies from 2023 to 2025 is approximately $2,192,060,000,000. Let me know if you want deeper insights. 



💬 You:  exit


👋 Bot: Catch you later!
